### **predict**

In [3]:
import cv2
import torch
from ultralytics import YOLO

# 載入 YOLOv10 模型
model = YOLO(r"yolov10-main\runs\detect\train\weights\best.pt")  # 載入模型

# 載入影片檔案
input_video_path = r'datasets\test\Walk in Shibuya_daytime.mp4'
output_video_path = r'datasets\test\output_video\Walk in Shibuya_daytime.mp4'  # 確認包含完整的檔案名稱和副檔名

# 使用 OpenCV 開啟影片
video_capture = cv2.VideoCapture(input_video_path)

# 獲取影片屬性
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_capture.get(cv2.CAP_PROP_FPS))
total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

# 定義編碼器並創建 VideoWriter 物件以保存輸出影片
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 編碼器
out_video = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# 遍歷每一幀
frame_count = 0
while video_capture.isOpened():
    ret, frame = video_capture.read()  # 讀取一幀
    if not ret:
        break
    
    # 應用 YOLOv10 物件偵測
    results = model(frame)[0]
    
    # 遍歷偵測結果並繪製邊界框
    for result in results.boxes.data.tolist():  # 每個偵測結果的格式為 [x1, y1, x2, y2, conf, class]
        x1, y1, x2, y2, conf, cls = result[:6]
        label = f'{model.names[cls]} {conf:.2f}'
        
        # 在幀上繪製邊界框和標籤
        if conf > 0.5: 
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 4)  # 邊界框
            # cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    
    # 將處理過的幀寫入輸出影片
    out_video.write(frame)

# 釋放資源
video_capture.release()
out_video.release()
cv2.destroyAllWindows()

print(f'Output video saved to {output_video_path}')


0: 384x640 5 faces, 85.0ms
Speed: 5.0ms preprocess, 85.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 faces, 12.0ms
Speed: 3.0ms preprocess, 12.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 faces, 12.0ms
Speed: 2.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 faces, 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 6 faces, 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 faces, 13.0ms
Speed: 2.0ms preprocess, 13.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 7 faces, 11.0ms
Speed: 2.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 faces, 10.0ms
Speed: 2.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


### **Real-time testing**

In [ ]:
import cv2
from ultralytics import YOLO

# 載入 YOLOv10 模型
model = YOLO(r"yolov10-main\runs\detect\train\weights\best.pt")  # 載入模型

#  使用攝像頭 
cap = cv2.VideoCapture(0)

# 遍歷每一幀
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # 使用 YOLOv10 模型進行物件偵測
    results = model(frame)

    # 遍歷偵測結果並繪製邊界框
    for result in results:
        for box in result.boxes:
            # 獲取邊界框座標、置信度和類別
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0]
            cls = box.cls[0]

            # 繪製邊界框和標籤
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = f'{model.names[int(cls)]}: {conf:.2f}'
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # 顯示結果
    cv2.imshow("YOLO Realtime Detection", frame)

    # 按 'q' 鍵退出
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 釋放資源
cap.release()
cv2.destroyAllWindows()
